<a href="https://colab.research.google.com/github/sudht/NLP/blob/master/week10_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>개인 구글 드라이브와 colab 연동</h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


<h2>RNN 모델</h2>

![실습 그림](http://nlp.kangwon.ac.kr/~nlpdemo/ner_figure.PNG)

In [2]:
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

class RNN(object):
  
  def __init__(self, flags):
    self.encoder_vocab_size = flags["encoder_vocab_size"]    # 음절 개수
    self.label_vocab_size = flags["label_vocab_size"]
    self.embedding_size = flags["embedding_size"]            # 음절 임베딩 사이즈
    self.hidden_size = flags["hidden_size"]                  # GRU 히든 사이즈

    self.max_length = flags["max_length"]                    # 음절 최대 길이
    self.keep_prob = flags["keep_prob"]                      # 노드를 보전할 확률
    self.learning_rate = flags["learning_rate"]              # 학습률
    self.mode = flags["mode"]                                # 학습 or 평가 상태

    self._input_init()
    self._embedding_init()
    self._rnn_init()
    self._train_init()
    self._predict_init()
        
  # 입력 데이터, 입력 데이터의 길이, 정답 데이터, keep_prob 값을 담을 tensor 선언  
  def _input_init(self):
    # 입력 데이터
    self.inputs = tf.placeholder(tf.int32, [None, self.max_length], name="inputs")
    
    # 입력 데이터의 길이
    self.inputs_length = tf.placeholder(tf.int32, [None], name="inputs_length")

    # 정답 데이터
    self.targets = tf.placeholder(tf.int32, [None, self.max_length], name="targets")
    # loss를 구할 때 각 sequence의 가중치를 설정
    self.target_weights = tf.sequence_mask(lengths=self.inputs_length, maxlen=self.max_length, dtype=tf.float32, name='weight')
    # 노드를 보전할 확률
    self.keep_prob = tf.placeholder(tf.float32, [], "keep_prob")

  def _embedding_init(self):
    # 입력 데이터에 있는 각 음절 index를 대응하는 임베딩 벡터로 치환해주기 위한 임베딩 tensor
    # 기존에 사전학습 된 음절 임베딩을 사용할 수도 있고 랜덤으로 초기화 한 후,
    # 모델 학습 과정 중에 같이 학습 시키는 것도 가능
    # 예제 코드는 랜덤으로 초기화 한 후 같이 학습하도록 설정
    self.encoder_embeddings = tf.get_variable("encoder_embedding",
                                             shape = [self.encoder_vocab_size, self.embedding_size],
                                             dtype = tf.float32, trainable=True,
                                             initializer=tf.random_normal_initializer(stddev=0.1))

  def _rnn_init(self):
    with tf.name_scope("rnn_layer"):
      # rnn에서 사용할 cell 설정
      # 예제 코드에서는 bidirectional lstm를 사용하기 때문에 정방향 cell과 역방향 cell 두개를 사용
      fw_cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size)
      fw_cell = tf.contrib.rnn.DropoutWrapper(fw_cell, input_keep_prob=self.keep_prob, output_keep_prob=self.keep_prob)
      bw_cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size)
      bw_cell = tf.contrib.rnn.DropoutWrapper(bw_cell, input_keep_prob=self.keep_prob, output_keep_prob=self.keep_prob)
      
      # 임베딩 tensor를 이용하여 입력데이터의 각 음절 index를 대응하는 임베딩 벡터로 치환
      # (batch_size, max_length) -> (batch_size, max_length, embedding_size)
      self.lookup_inputs = tf.nn.embedding_lookup(self.encoder_embeddings, self.inputs)

      # tf.nn.bidirectional_dynamic_rnn 라이브러리를 사용하여 rnn layer의 출력을 구한다
      # outputs : 각 step의 rnn 출력값, state : 마지막 hidden state 값
      (fw_outputs, bw_outputs), (fw_state, bw_state) = tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, self.lookup_inputs,
                                                                                       sequence_length=self.inputs_length,
                                                                                       dtype=tf.float32,
                                                                                       time_major=False)

      # 예제 코드에서는 각 step의 출력값을 사용한다.
      # 양방향으로 생성된 rnn 결과를 연결하여 하나의 벡터로 표현
      # (batch_size, max_length, hidden_size)*2 -> (batch_size, max_length, hidden_size*2)
      outputs = tf.concat([fw_outputs, bw_outputs], axis=2)
      
      # fully_connected layer를 통하여 출력 크기를 label_vocab_size에 맞춰줌
      # (batch_size, max_length, hidden_size*2) -> (batch_size, max_length, label_vocab_size)
      self.outputs = tf.layers.dense(inputs=outputs,
                                     units=self.label_vocab_size,
                                     activation=tf.nn.relu,
                                     use_bias=False,
                                     kernel_initializer=tf.random_normal_initializer(stddev=0.01))
    
    
  def _predict_init(self):
    # 각 라벨의 분포 값들을 softmax 함수를 이용하여 0~1사이의 값으로 변경
    self.predict_op = tf.nn.softmax(logits=self.outputs, axis=-1)

  def _train_init(self):
    if self.mode == "train":
      with tf.variable_scope("train_layer"):
        # 모델의 출력인 self.outputs 와 self.targets를 비교하여 loss 계산
        self.loss = tf.contrib.seq2seq.sequence_loss(self.outputs, self.targets, self.target_weights)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.grads = self.optimizer.compute_gradients(self.loss)
        self.train_op = self.optimizer.apply_gradients(self.grads)

<h2>데이터 읽고 전처리 하기</h2>

<pre>
<b>1. read_file(file_path)</b>
  "train_datas.txt", "test_datas.txt" 파일을 읽기 위한 함수
  
  데이터 예시)
    대 한 민 국 은 〈SP〉민 주 주 의 국 가 이 다 . \t B_OG I I I O 〈SP〉O O O O O O O O O
  
  read_file(file_path)
  args
    file_path : 읽고자 하는 데이터의 경로
  return
    음절 sequence, 음절 sequence의 길이, 각 음절에 대응하는 라벨 sequence를 담고 있는 리스트
    
    출력 예시)
      ["대", "한", "민", "국", "은", "〈SP〉",  "민", "주", "주", "의", "국", "가", "이", "다", "."], 15, ["B_OG", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O"]
      
<b>2. read_vocab_file(flags)</b>
  "eumjeol_vocab.txt", "label_vocab.txt" 파일을 읽고 음절과 라벨을 indexing하기 위한 딕셔너리를 생성
   
  read_vocab_file(flags)
  args
    flags : hyperparameter들을 담고 있는 딕셔너리
  return
    eumjeol2idx, idx2eumjeol, label2idx, idx2label 딕셔너리
    
    eumjeol2idx : 음절을 대응하는 index로 치환하기 위한 딕셔너리
    idx2eumjeol : index를 대응하는 음절로 치환하기 위한 딕셔너리
    label2idx : 라벨을 대응하는 index로 치환하기 위한 딕셔너리
    idx2label : index를 대응하는 라벨로 치환하기 위한 딕셔너리
    
<b>3. prepro(datas, eumjeol2idx, label2idx)</b>
  입력 데이터의 각 음절과 라벨을 indexing, 음절 sequence의 길이를 고정된 길이로 변환
   
  prepro(datas, eumjeol2idx, label2idx)
  args
    file_path : 음절 sequence, 음절 sequence의 길이, 각 음절에 대응하는 라벨 sequence를 담고 있는 리스트
  return
    입력 데이터의 각 음절을 indexing하고 라벨을 one_hot 인코딩한 리스트
    
  전처리 예시)
    ["대", "한", "민", "국", "은", "〈SP〉",  "민", "주", "주", "의", "국", "가", "이", "다", "."], 15, ["B_OG", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O"]
    
    사전 설정한 문장의 최대 길이를 20이라고 가정
    
     ["대", "한", "민", "국", "은", "〈SP〉",  "민", "주", "주", "의", "국", "가", "이", "다", ".", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉"]
         ["대", "한", "민", "국", "은", "〈SP〉",  "민", "주", "주", "의", "국", "가", "이", "다", ".", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉"] -> [ 23, 2, 55, 65, 96, 12, 4, 11, 235, 5, 3, 12, 98, 1, 0, 0, 0, 0, 0, 0]
    
    ["B_OG", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O"] -> ["B_OG", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉"]
    ["B_OG", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉", "〈PAD〉"] -> [ 7, 2, 2, 2, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0 ]    
 </pre>
        
  


In [0]:
import numpy as np

# 데이터를 읽고 음절 sequence와 라벨 sequence를 분리
# 음절 sequence의 길이를 구하고 음절 sequence, 음절 sequence의 길이, 라벨 sequence를 datas 리스트에 저장
def read_file(file_path):
  with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()
  datas = []
  for line in lines:
    # 입력 문장을 \t으로 분리
    pieces = line.strip().split("\t")
    eumjeol_sequence, label_sequence = pieces[0].split(), pieces[1].split()
    eumjeol_length = len(eumjeol_sequence)

    datas.append((eumjeol_sequence, eumjeol_length, label_sequence))

  return datas
  
def read_vocab_file(flags):
  eumjeol2idx, idx2eumjeol = {"<PAD>":0}, {0:"<PAD>"}
  label2idx, idx2label = {"<PAD>":0}, {0:"<PAD>"}
  
  with open(flags["eumjeol_vocab_data_path"], "r", encoding="utf8") as inFile:
    eumjeols = inFile.readlines()
  with open(flags["label_vocab_data_path"], "r", encoding="utf8") as inFile:
    labels = inFile.readlines()

  for eumjeol in eumjeols:
    eumjeol = eumjeol.strip()

    if(eumjeol not in eumjeol2idx):
      eumjeol2idx[eumjeol] = len(eumjeol2idx)

    if(eumjeol2idx[eumjeol] not in idx2eumjeol):
      idx2eumjeol[eumjeol2idx[eumjeol]] = eumjeol

  for label in labels:
    label = label.strip()

    if(label not in label2idx):
      label2idx[label] = len(label2idx)

    if(label2idx[label] not in idx2label):
      idx2label[label2idx[label]] = label
      
  return eumjeol2idx, idx2eumjeol, label2idx, idx2label
        
# 입력 데이터 전처리 
def prepro(datas, eumjeol2idx, label2idx, max_length):
  
  preprocessed_datas = []
  for eumjeol_sequence, eumjeol_length, label_sequence in datas:
    
    # max_length의 길이를 가지는 numpy array 생성하고 0으로 초기화
    indexing_eumjeol_sequence = np.zeros(max_length, dtype=np.int32)
    indexing_label_sequence = np.zeros(max_length, dtype=np.int32)
    # 각 음절과 라벨 index를 numpy array에 입력
    for index in range(eumjeol_length):
      indexing_eumjeol_sequence[index] = eumjeol2idx[eumjeol_sequence[index]]
      indexing_label_sequence[index] = label2idx[label_sequence[index]]
    preprocessed_datas.append((indexing_eumjeol_sequence, eumjeol_length, indexing_label_sequence))
  return preprocessed_datas

<h2>데이터 batch 단위로 나누기</h2>

<pre>
<b>get_batch(datas, batch_size)</b>
  전체 데이터를 batch 단위로 나누어 주기 위한 함수
  
  get_batch(datas, batch_size)
  args
    datas : 음절 sequence, 음절 sequence의 길이, 각 음절에 대응하는 라벨 sequence를 담고 있는 리스트
    batch_size : 한번에 학습할 데이터의 개수
  return
    batch 단위로 나뉘어진 데이터 리스트
    
  예시) 
    batch_size = 3 인 경우
  
    total_datas = [ (음절 sequence1, sequence1의 길이, 라벨 sequence1), ... , ((음절 sequence100, sequence100의 길이, 라벨 sequence100)) ]
    
    batches = [
    [ [ 음절 sequence1, 음절 sequence2, 음절 sequence3 ], 
      [ sequence1의 길이, sequence2의 길이, sequence3의 길이 ],
      [ 라벨 sequence1, 라벨 sequence2, 라벨 sequence3 ]
    ],
    
    [ [ 음절 sequence4, 음절 sequence5, 음절 sequence6 ], 
      [ sequence4의 길이, sequence5의 길이, sequence6의 길이 ],
      [ 라벨 sequence4, 라벨 sequence5, 라벨 sequence6 ]
    ],
    
    ...
    
    ]

</pre>

In [0]:
# 데이터를 batch 단위로 분할하여 저장
def get_batch(datas, batch_size):
  # batches : batch 단위로 저장할 리스트
  # inputs : 각 batch 단위 별 음절 sequence를 저장할 리스트
  # inputs_length : 각 batch 단위 별 음절 sequence 길이를 저장할 리스트
  # targets : 각 batch 단위 별 라벨 sequence를 저장할 리스트
  batches, inputs, inputs_length, targets = [], [], [], []
    
  for indexing_eumjeol_sequence, eumjeol_length, indexing_label_sequence in datas:
    inputs.append(indexing_eumjeol_sequence)
    inputs_length.append(eumjeol_length)
    targets.append(indexing_label_sequence)
    
    if(len(inputs) == batch_size):
      batches.append((inputs, inputs_length, targets))
      inputs, inputs_length, targets = [], [], []

  return batches

<h2>RNN 모델 학습</h2>

<pre>
<b>1. read_vocab_file(flags) 함수를 사용하여 딕셔너리 생성</b>

<b>2. read_file(file_path) 함수를 사용하여 학습 데이터 읽기</b>

<b>3. prepro 함수를 사용하여 학습 데이터 전처리</b>

<b>4. RNN 모델 객체 선언</b>

<b>3. epoch를 돌때마다 학습 데이터 셔플</b>

<b>3. batch 단위로 학습을 수행</b>
</pre>

In [0]:
import os
import numpy as np
from tqdm import tqdm

def train(flags):
  # vocab.txt 파일을 읽고 딕셔너리 생성
  eumjeol2idx, idx2eumjeol, label2idx, idx2label = read_vocab_file(flags)
  # 학습 데이터 읽기
  train_datas = read_file(flags["train_data_path"])
  # 학습 데이터 전처리
  preprocessed_train_datas = prepro(train_datas, eumjeol2idx, label2idx, flags["max_length"])
  
  # 모델 객체 선언
  model = RNN(flags)
  
  # tensorflow session 옵션 설정
  # allow_soft_placement=True : 어떤 device를 사용하여 연산할지 명시하지 않은 경우 자동으로 존재하는 디바이스 중에서 하나를 선택
  # gpu_options=tf.GPUOptions(allow_growth=True) : 연산 실행 과정에서 필요한만큼의 gpu 메모리만 사용
  sess_config = tf.ConfigProto(allow_soft_placement=True, 
                               gpu_options=tf.GPUOptions(allow_growth=True))

  # tensorflow를 실행하기 위한 session
  with tf.Session(config=sess_config) as sess:
    # 그래프 초기화
    sess.run(tf.global_variables_initializer())
    # 학습 파일을 저장거나 불러오기 위한 saver 객체
    saver = tf.train.Saver()

    for epoch in tqdm(range(flags["epoch"])):
      # 학습 데이터 셔플
      np.random.shuffle(preprocessed_train_datas)
      # 학습 데이터를 batch 단위로 분할하여 저장
      batches = get_batch(preprocessed_train_datas, flags["batch_size"])

      losses = []
      # batch 단위로 학습을 진행하며 각 batch 별 loss를 구한다
      # batch 별 loss들의 평균을 구하여 이를 전체 데이터에 대한 loss로 사용
      for inputs, inputs_length, targets in batches:
        loss, train_op = sess.run([model.loss, model.train_op],
                                  feed_dict={ model.inputs:inputs, 
                                             model.inputs_length:inputs_length, 
                                             model.targets:targets, 
                                             model.keep_prob:flags["keep_prob"] }
                                 )

        losses.append(loss)
        
      # 학습한 모델 파일 저장
      filename = os.path.join(flags["save_dir"], "model_{}.ckpt".format(epoch+1))
      saver.save(sess, filename)

      print("\tEpoch : {}, Average_Loss : {}".format(epoch+1, np.mean(losses)))

<h2>RNN 모델 평가</h2>

<pre>
<b>1. read_vocab_file(flags) 함수를 사용하여 딕셔너리 생성</b>

<b>2. read_file(file_path) 함수를 사용하여 평가 데이터 읽기</b>

<b>3. prepro 함수를 사용하여 평가 데이터 전처리</b>

<b>4. RNN 모델 객체 선언</b>

<b>5. tf.train.Saver() 객체를 사용하여 학습한 모델 파일 중에서 가장 많이 학습된 파일로부터 가중치를 불러옴</b>

<b>6. make_sentence(inputs, predict, correct, idx2eumjeol, idx2label) 함수를 이용하여 정답과 모델 출력 비교</b>
  
  make_sentence(inputs, predict, correct, idx2eumjeol, idx2label)
  args
    inputs : 음절 sequence
    predict : 모델 출력 라벨 sequence
    correct : 정답 라벨 sequence
    idx2eumjeol : index를 대응하는 음절로 치환해주는 딕셔너리
    idx2label : index를 대응하는 라벨로 치환해주는 딕셔너리
  return
    정답 문장과 모델 출력 문장

  예시)
    inputs = ["대", "한", "민", "국", "은", "〈SP〉",  "민", "주", "주", "의", "국", "가", "이", "다", "."]
    predict = ["B_LC", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O"]
    correct = ["B_OG", "I", "I", "I", "O", "〈SP〉", "O", "O", "O", "O", "O", "O", "O", "O", "O"]
    
    정답 문장 : <대한민국:OG>은 민주주의국가이다.
    모델 출력 문장 : <대한민국:LC>은 민주주의 국가이다.

</pre>

In [0]:
import numpy as np
from tqdm import tqdm

# 모델 출력 라벨 sequence와 정답 라벨 sequence를 기반으로
# 모델 출력 문장과 정답 문장 출력
def make_sentence(inputs, label_idx_list, idx2eumjeol, idx2label):
  
  # 빈 문자열 생성
  flag = False
  restored_sentence = ""
  ner = ""
  tag = ""
  for index in range(len(label_idx_list)):
    eumjeol = idx2eumjeol[inputs[index]]
    label = idx2label[label_idx_list[index]]
    if label[0] == 'B':
      #####################################################
      flag = True
      restored_sentence += ("<" + eumjeol)
      tag += ":" + label[2] + label[3] + ">"
      #####################################################

    elif label == 'I':
      #####################################################
      restored_sentence += eumjeol
      #####################################################

    elif label == 'O':
      #####################################################
      if flag == True:
        restored_sentence += tag
        flag = False
        tag = ""
      restored_sentence += eumjeol
      #####################################################

    else:
      #####################################################
      if flag == True:
        restored_sentence += tag
        flag = False
        tag = ""
      restored_sentence += " "
      #####################################################
  return restored_sentence
      

def test(flags):
  # vocab.txt 파일을 읽고 딕셔너리 생성
  eumjeol2idx, idx2eumjeol, label2idx, idx2label = read_vocab_file(flags)
  # 평가 데이터 읽기
  test_datas = read_file(flags["test_data_path"])
  # 평가 데이터 전처리
  preprocessed_test_datas = prepro(test_datas, eumjeol2idx, label2idx, flags["max_length"])
  
  # 모델 객체 선언
  model = RNN(flags)
  # tensorflow session 옵션 설정
  sess_config = tf.ConfigProto(allow_soft_placement=True, 
                               gpu_options=tf.GPUOptions(allow_growth=True))

  # tensorflow를 실행하기 위한 session
  with tf.Session(config=sess_config) as sess:
    # 그래프 초기화
    sess.run(tf.global_variables_initializer())
    # 학습 파일을 저장거나 불러오기 위한 saver 객체
    saver = tf.train.Saver()
    
    # 학습한 모델 파일 중에서 가장 많이 학습된 파일로부터 가중치를 불러옴
    print("Read from : " + str(tf.train.latest_checkpoint(flags["save_dir"])))
    saver.restore(sess, tf.train.latest_checkpoint(flags["save_dir"]))

    # np.random.shuffle(preprocessed_test_datas)

    # 평가 데이터를 batch 단위로 분할하여 저장
    batches = get_batch(preprocessed_test_datas, flags["batch_size"])

    # 전체 음절 수, 정답을 맞춘 음절 수
    total_count, correct_count = 0, 0
    for inputs, inputs_length, targets in tqdm(batches[:10]):
      predict_op = sess.run(model.predict_op,
                            feed_dict={ model.inputs:inputs, 
                                       model.inputs_length:inputs_length, 
                                       model.keep_prob:flags["keep_prob"] }
                           )
      
      # 모델의 outputs에는 각 클래스에 대한 분포가 저장되어 있고
      # np.argmax 함수를 통하여 가장 확률이 높은 클래스를 선택
      # 예시) 
      #  predict_op = [0,1, 0.3, 0.2] (각각 0일 확률, 1일 확률, 2일 확률)
      #  np.argmax(predict_op) = 1
      predict, correct = np.argmax(predict_op[0], axis=-1), targets[0]
      
      # padding 처리해준 부분 제거
      predict, correct = np.trim_zeros(predict), np.trim_zeros(correct)
      
      # print([idx2eumjeol[temp] for temp in inputs[0]])
      # print([idx2label[temp] for temp in predict])
      # print([idx2label[temp] for temp in correct])

      predict_sentence = make_sentence(inputs[0], predict, idx2eumjeol, idx2label)
      correct_sentence = make_sentence(inputs[0], correct, idx2eumjeol, idx2label)
      print("\n정답 : " + correct_sentence)
      print("출력 : " + predict_sentence + "\n")
      
      correct_count += np.sum(np.equal(predict, correct))
      total_count += inputs_length[0]

    print("Accuracy : " + str(100.0*correct_count/total_count))

<h2>모델의 hyper parameter 설정, 학습 및 평가 실행</h2>

<pre>
root_dir : 코드와 데이터가 있는 디렉토리 경로
save_dir : 학습한 모델 파일을 저장할 디렉토리 경로(디렉토리가 존재하지 않을 경우 자동으로 생성)

<b>flags : hyper parameter를 저장할 딕셔너리</b>
  flags.mode = 학습 또는 평가 설정("train" or "test")
  flags.save_dir = 학습한 모델 파일을 저장할 디렉토리 경로
  flags.batch_size = 한번에 학습할 데이터의 개수
  flags.epoch = 학습 횟수
  flags.learning_rate = 학습률
  flags.keep_prob = 노드를 보전할 확률
  flags.max_length = 음절 sequence의 최대 길이
  flags.embedding_size = 음절 임베딩 사이즈
  flags.hidden_size = rnn cell의 히든 사이즈
  flags.encoder_vocab_size = 음절 어휘 딕셔너리의 사이즈
  flags.label_vocab_size = 라벨 딕셔너리의 사이즈
  flags.train_data_path = 학습데이터 파일 경로
  flags.test_data_path = 평가데이터 파일 경로
  flags.vocab_data_path = 음절 어휘 파일 경로

<b>mode 별 hyper parameter 변경</b>
  학습하는 경우 : mode를 "train"으로 설정, 나머지는 기본 설정 그대로 유지
  평가하는 경우 : mode를 "test"로, batch_size는 1로, keep_prob은 1.0으로 변경
</pre>

In [8]:
import os
import tensorflow as tf

if __name__ == "__main__":
  root_dir = "/gdrive/My Drive/colab/week10"
  save_dir = os.path.join(root_dir, "model")
  if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  flags = {"mode":"test",
           "save_dir":save_dir,
           "batch_size":1,
           "epoch":100,
           "learning_rate":0.0005,
           "keep_prob":1.0,
           "max_length":100,
           "embedding_size":100,
           "hidden_size":100,
           "encoder_vocab_size":2023,
           "label_vocab_size":9,
           "train_data_path":os.path.join(root_dir, "ner_train.txt"),
           "test_data_path":os.path.join(root_dir, "ner_test.txt"),
           "eumjeol_vocab_data_path":os.path.join(root_dir, "vocab.txt"),
           "label_vocab_data_path":os.path.join(root_dir, "label_vocab.txt")
          }
  
  tf.reset_default_graph()
  if(flags["mode"] == "train"):
      train(flags)
  elif(flags["mode"] == "test"):
      flags["batch_size"] = 1  
      flags["keep_prob"] = 1.0
      test(flags)
  else:
      print("Unknown mode")
      exit(0)

  0%|          | 0/10 [00:00<?, ?it/s]

Read from : /gdrive/My Drive/colab/week10/model/model_100.ckpt


 10%|█         | 1/10 [00:00<00:02,  4.07it/s]


정답 : <근대:DT> 이후 <현대:DT>까지 내려오는 <서양:LC>의 人間觀은 물론 사람과 시대에 따라 여러 가지 있었지만 대체로 이 <르네상스:DT> 인간관을 그 基調로 하고 세워진 것이라고 볼 수 있다.
출력 : 근대 이후 <현대:PS>까지 내려오는 서양의 人間觀은 물론 사람과 시대에 따라 여러 가지 있었지만 대체로 이 르네상스 인간관을 그 基調로 하고 세워진 것이라고 볼 수 있다.


정답 : 어느 날 <현우:PS>는 뜰 끝에 나와 앉아서 동무한테서 빌린 중학 강의록 책을 보고 있었다.
출력 : 어느 날 <현우:PS>는 뜰 끝에 나와 앉아서 동무한테서 빌린 중학 강의록 책을 보고 있었다.



 50%|█████     | 5/10 [00:00<00:00,  5.97it/s]


정답 : 그는 멍청한 <우디:PS>를 부사장으로 승진시키고, 폭력배인 케빈 스페이시 일당을 통해 공장 설비를 파손시키고 그 책임을 물어 노동자들을 해고하려 한다.
출력 : 그는 멍청한 우디를 부사장으로 승진시키고, 폭력배인 케빈 스페이시 일당을 통해 공장 설비를 파손시키고 그 책임을 물어 노동자들을 해고하려 한다.


정답 : 누이동생의 말에 부시시 일어난 <수창이:PS> 아버지는 크게 하품을 하고 나서 힘없이 말한다.
출력 : 누이동생의 말에 부시시 일어난 <수창이:PS> 아버지는 크게 하품을 하고 나서 힘없이 말한다.


정답 : 그리고 물질과 그것의 운동, 이 두 가지 요소가 바로 <데카르트:PS>의 기계적 철학의 근본요소가 되었다.
출력 : 그리고 물질과 그것의 운동, 이 두 가지 요소가 바로 <데카르트:PS>의 기계적 철학의 근본요소가 되었다.



 70%|███████   | 7/10 [00:00<00:00,  7.13it/s]


정답 : <탁:PS>선생의 말이 알듯 모를 듯 싶었지만 좌우지간 <영애:PS>는 가수만 되면 되는 거였다.
출력 : <탁:PS>선생의 말이 알듯 모를 듯 싶었지만 좌우지간 <영애:PS>는 가수만 되면 되는 거였다.


정답 : <박:PS>기자 옆에 누워서 아삭아삭 건빵을 씹어먹던 <김:PS>일병이 빨딱 일어나 앉았습니다.
출력 : 박기자 옆에 누워서 아삭아삭 건빵을 씹어먹던 <김일:PS>병이 빨딱 일어나 앉았습니다.


정답 : <鄭夢周:PS>를 때려 죽였는데도 불구하고 아직도 <善竹橋:LC>에 피가 흐른다는 것은 무엇인가?
출력 : <鄭夢周:PS>를 때려 죽였는데도 불구하고 아직도 善竹橋에 피가 흐른다는 것은 무엇인가?



100%|██████████| 10/10 [00:01<00:00,  7.88it/s]


정답 : 따라서 <일년:DT>이 비록 넉넉하지는 못할지라도 반드시 무슨 초월적인 힘에 의지해야만 그가 원하는 바를 알 수 있게 될 만큼의 짧은 기간 또한 아니었다.
출력 : 따라서 일년이 비록 넉넉하지는 못할지라도 반드시 무슨 초월적인 힘에 의지해야만 그가 원하는 바를 알 수 있게 될 만큼의 짧은 기간 또한 아니었다.


정답 : <정부:OG>는 잠수함 침투사건을 계기로 <북한:LC>의 실체를 새롭게 인식하고 <북한:LC>에 대해 더 이상 양보가 없는 강경한 자세로 전환하고 있다.
출력 : <정부:PS>는 잠수함 침투사건을 계기로 북한의 실체를 새롭게 인식하고 <북한:PS>에 대해 더 이상 양보가 없는 강경한 자세로 전환하고 있다.

Accuracy : 96.33943427620632
